In [111]:
!pip install category_encoders

In [112]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from scipy.stats import boxcox
from sklearn.preprocessing import StandardScaler
from category_encoders import TargetEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error


from sklearn.metrics import (
    mean_squared_error,     # MSE
    mean_absolute_error,    # MAE
    r2_score                # R2 Error
)
from sklearn.model_selection import cross_val_score, KFold

# Warnings Library
import warnings                      # Main Library
warnings.filterwarnings("ignore")

In [113]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [114]:
df = pd.read_csv('/content/drive/MyDrive/country/death .csv')

# **Basic EDA checks**

In [115]:
df.head()

,index,County,FIPS,Met Objective of 45.5? (1),Age-Adjusted Death Rate,Lower 95% Confidence Interval for Death Rate,Upper 95% Confidence Interval for Death Rate,Average Deaths per Year,Recent Trend (2),Recent 5-Year Trend (2) in Death Rates,Lower 95% Confidence Interval for Trend,Upper 95% Confidence Interval for Trend
0,0,United States,0,No,46,45.9,46.1,"157,376",falling,-2.4,-2.6,-2.2
1,1,"Perry County, Kentucky",21193,No,125.6,108.9,144.2,43,stable,-0.6,-2.7,1.6
2,2,"Powell County, Kentucky",21197,No,125.3,100.2,155.1,18,stable,1.7,0,3.4
3,3,"North Slope Borough, Alaska",2185,No,124.9,73,194.7,5,**,**,**,**
4,4,"Owsley County, Kentucky",21189,No,118.5,83.1,165.5,8,stable,2.2,-0.4,4.8


In [116]:
df.columns

Index(['index', 'County', 'FIPS', 'Met Objective of 45.5? (1)',
       'Age-Adjusted Death Rate',
       'Lower 95% Confidence Interval for Death Rate',
       'Upper 95% Confidence Interval for Death Rate',
       'Average Deaths per Year', 'Recent Trend (2)',
       'Recent 5-Year Trend (2) in Death Rates',
       'Lower 95% Confidence Interval for Trend',
       'Upper 95% Confidence Interval for Trend'],
      dtype='object')

In [117]:
df.shape

(3141, 12)

In [118]:
df.count()

,0
index,3141
County,3141
FIPS,3141
Met Objective of 45.5? (1),3141
Age-Adjusted Death Rate,3141
Lower 95% Confidence Interval for Death Rate,3141
Upper 95% Confidence Interval for Death Rate,3141
Average Deaths per Year,3141
Recent Trend (2),3141
Recent 5-Year Trend (2) in Death Rates,3141


In [119]:
df.describe(include='all')

,index,County,FIPS,Met Objective of 45.5? (1),Age-Adjusted Death Rate,Lower 95% Confidence Interval for Death Rate,Upper 95% Confidence Interval for Death Rate,Average Deaths per Year,Recent Trend (2),Recent 5-Year Trend (2) in Death Rates,Lower 95% Confidence Interval for Trend,Upper 95% Confidence Interval for Trend
count,3141.000000,3141,3141.000000,3141,3141,3141,3141,3141,3141,3141,3141,3141
unique,NaN,3141,NaN,3,618,572,767,307,5,149,155,129
top,NaN,"Ziebach County, South Dakota",NaN,No,*,*,*,*,stable,**,**,**
freq,NaN,1,NaN,1985,328,328,328,328,1796,420,420,420
mean,1570.000000,NaN,30392.281757,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,906.872924,NaN,15158.200839,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,0.000000,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,785.000000,NaN,18179.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,1570.000000,NaN,29177.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,2355.000000,NaN,45081.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [120]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3141 entries, 0 to 3140
Data columns (total 12 columns):
 #   Column                                        Non-Null Count  Dtype 
---  ------                                        --------------  ----- 
 0   index                                         3141 non-null   int64 
 1   County                                        3141 non-null   object
 2   FIPS                                          3141 non-null   int64 
 3   Met Objective of 45.5? (1)                    3141 non-null   object
 4   Age-Adjusted Death Rate                       3141 non-null   object
 5   Lower 95% Confidence Interval for Death Rate  3141 non-null   object
 6   Upper 95% Confidence Interval for Death Rate  3141 non-null   object
 7   Average Deaths per Year                       3141 non-null   object
 8   Recent Trend (2)                              3141 non-null   object
 9   Recent 5-Year Trend (2) in Death Rates        3141 non-null   object
 10  

In [121]:
df.duplicated().any()

np.False_

In [122]:
print(repr(df.columns))

Index(['index', 'County', 'FIPS', 'Met Objective of 45.5? (1)',
       'Age-Adjusted Death Rate',
       'Lower 95% Confidence Interval for Death Rate',
       'Upper 95% Confidence Interval for Death Rate',
       'Average Deaths per Year', 'Recent Trend (2)',
       'Recent 5-Year Trend (2) in Death Rates',
       'Lower 95% Confidence Interval for Trend',
       'Upper 95% Confidence Interval for Trend'],
      dtype='object')


# **Data cleaning **

In [123]:
if df.isnull().values.any():
  print("NaN present")
else:
  print("No NaN values")

No NaN values


In [124]:
for col in df.columns:
    print(f"{col} has {df[col].isna().sum()} empty values")

index has 0 empty values
County has 0 empty values
FIPS has 0 empty values
Met Objective of 45.5? (1) has 0 empty values
Age-Adjusted Death Rate has 0 empty values
Lower 95% Confidence Interval for Death Rate has 0 empty values
Upper 95% Confidence Interval for Death Rate has 0 empty values
Average Deaths per Year has 0 empty values
Recent Trend (2) has 0 empty values
Recent 5-Year Trend (2) in Death Rates has 0 empty values
Lower 95% Confidence Interval for Trend has 0 empty values
Upper 95% Confidence Interval for Trend has 0 empty values


# **Feature target and splitting **

In [125]:
# Target column
y = df['Recent Trend (2)']

# Features (drop the target column)
X = df.drop(['Recent Trend (2)'], axis=1)

In [126]:
y

,Recent Trend (2)
0,falling
1,stable
2,stable
3,**
4,stable
...,...
3136,**
3137,**
3138,*
3139,**


In [127]:
X

,index,County,FIPS,Met Objective of 45.5? (1),Age-Adjusted Death Rate,Lower 95% Confidence Interval for Death Rate,Upper 95% Confidence Interval for Death Rate,Average Deaths per Year,Recent 5-Year Trend (2) in Death Rates,Lower 95% Confidence Interval for Trend,Upper 95% Confidence Interval for Trend
0,0,United States,0,No,46,45.9,46.1,"157,376",-2.4,-2.6,-2.2
1,1,"Perry County, Kentucky",21193,No,125.6,108.9,144.2,43,-0.6,-2.7,1.6
2,2,"Powell County, Kentucky",21197,No,125.3,100.2,155.1,18,1.7,0,3.4
3,3,"North Slope Borough, Alaska",2185,No,124.9,73,194.7,5,**,**,**
4,4,"Owsley County, Kentucky",21189,No,118.5,83.1,165.5,8,2.2,-0.4,4.8
...,...,...,...,...,...,...,...,...,...,...,...
3136,3136,"Yakutat City and Borough, Alaska<sup>3</sup>",2282,*,*,*,*,*,**,**,**
3137,3137,"Yukon-Koyukuk Census Area, Alaska",2290,*,*,*,*,*,**,**,**
3138,3138,"Zapata County, Texas",48505,*,*,*,*,*,*,*,*
3139,3139,"Zavala County, Texas",48507,*,*,*,*,*,**,**,**


In [128]:
X.columns

Index(['index', 'County', 'FIPS', 'Met Objective of 45.5? (1)',
       'Age-Adjusted Death Rate',
       'Lower 95% Confidence Interval for Death Rate',
       'Upper 95% Confidence Interval for Death Rate',
       'Average Deaths per Year', 'Recent 5-Year Trend (2) in Death Rates',
       'Lower 95% Confidence Interval for Trend',
       'Upper 95% Confidence Interval for Trend'],
      dtype='object')

# **Train_test_split**

In [129]:
X_train, X_test, y_train, y_test = train_test_split(
    X, # Use the DataFrame with encoded columns
    y,
    test_size=0.2,
    random_state=42
)

In [130]:
X_train.shape

(2512, 11)

In [131]:
X_train

,index,County,FIPS,Met Objective of 45.5? (1),Age-Adjusted Death Rate,Lower 95% Confidence Interval for Death Rate,Upper 95% Confidence Interval for Death Rate,Average Deaths per Year,Recent 5-Year Trend (2) in Death Rates,Lower 95% Confidence Interval for Trend,Upper 95% Confidence Interval for Trend
2302,2302,"Dakota County, Minnesota",27037,Yes,40.8,37.8,43.9,148,-1,-1.5,-0.5
761,761,"Early County, Georgia",13099,No,60.1,43.2,81.9,8,1.2,-0.6,3
2194,2194,"Laramie County, Wyoming",56021,Yes,42.3,36.7,48.6,42,-0.4,-1.7,0.8
1241,1241,"Alpena County, Michigan",26007,No,54.3,45.1,65.1,25,-0.6,-1.6,0.4
221,221,"Fayette County, West Virginia",54019,No,73.5,64.3,83.8,47,-0.2,-1.2,0.8
...,...,...,...,...,...,...,...,...,...,...,...
3092,3092,"Southeast Fairbanks Census Area, Alaska",2240,*,*,*,*,*,**,**,**
1095,1095,"Houston County, Alabama",1069,No,56.1,50.3,62.4,69,-1.1,-1.7,-0.5
1130,1130,"Stanly County, North Carolina",37167,No,55.8,48.5,63.8,43,-0.3,-1.3,0.8
1294,1294,"Emmet County, Michigan",26047,No,53.8,44.6,64.6,25,-0.6,-2,0.8


In [132]:
X_test.shape

(629, 11)

In [133]:
X_test

,index,County,FIPS,Met Objective of 45.5? (1),Age-Adjusted Death Rate,Lower 95% Confidence Interval for Death Rate,Upper 95% Confidence Interval for Death Rate,Average Deaths per Year,Recent 5-Year Trend (2) in Death Rates,Lower 95% Confidence Interval for Trend,Upper 95% Confidence Interval for Trend
2894,2894,"Dimmit County, Texas",48127,*,*,*,*,*,**,**,**
2969,2969,"Jackson County, Colorado",8057,*,*,*,*,*,**,**,**
1064,1064,"Bedford County, Tennessee",47003,No,56.4,47.3,66.8,28,-0.8,-2,0.4
1231,1231,"Washington County, Idaho",16087,No,54.4,38.6,75.4,8,-1.1,-2.7,0.5
2988,2988,"Kittson County, Minnesota",27069,*,*,*,*,*,**,**,**
...,...,...,...,...,...,...,...,...,...,...,...
670,670,"Orange County, Texas",48361,No,61.9,54.9,69.4,59,-1.8,-2.8,-0.9
2283,2283,"Merced County, California",6047,Yes,41.1,37.3,45.3,85,-2.2,-2.8,-1.6
2482,2482,"Las Animas County, Colorado",8071,Yes,37.3,26.6,51.4,8,0.7,-1.7,3.1
2650,2650,"Richland County, North Dakota",38077,Yes,33.2,22.5,47.5,6,-0.2,-2.2,1.7


In [134]:
y_train

,Recent Trend (2)
2302,falling
761,stable
2194,stable
1241,stable
221,stable
...,...
3092,**
1095,falling
1130,stable
1294,stable


In [135]:
y_train.shape

(2512,)

In [136]:
y_test

,Recent Trend (2)
2894,**
2969,**
1064,stable
1231,stable
2988,**
...,...
670,falling
2283,falling
2482,stable
2650,stable


In [137]:
y_test.shape

(629,)

# **dtype conversion **

In [138]:
print(X_train.dtypes)

index                                            int64
County                                          object
FIPS                                             int64
Met Objective of 45.5? (1)                      object
Age-Adjusted Death Rate                         object
Lower 95% Confidence Interval for Death Rate    object
Upper 95% Confidence Interval for Death Rate    object
Average Deaths per Year                         object
Recent 5-Year Trend (2) in Death Rates          object
Lower 95% Confidence Interval for Trend         object
Upper 95% Confidence Interval for Trend         object
dtype: object


In [139]:
numeric_cols = X_train.select_dtypes(include=['object', 'int64', 'float64']).columns

for col in numeric_cols:
    X_train[col] = pd.to_numeric(X_train[col], errors='coerce')
    X_test[col] = pd.to_numeric(X_test[col], errors='coerce')

In [140]:
# Convert only numeric columns safely
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

# 3️⃣ Handle missing numeric values
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())

In [141]:
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)
y_train = y_train.fillna(0)
y_test = y_test.fillna(0)

In [142]:
print(X_train.dtypes)

index                                             int64
County                                          float64
FIPS                                              int64
Met Objective of 45.5? (1)                      float64
Age-Adjusted Death Rate                         float64
Lower 95% Confidence Interval for Death Rate    float64
Upper 95% Confidence Interval for Death Rate    float64
Average Deaths per Year                         float64
Recent 5-Year Trend (2) in Death Rates          float64
Lower 95% Confidence Interval for Trend         float64
Upper 95% Confidence Interval for Trend         float64
dtype: object


# **Encoding**

In [143]:
# Encode the target variable
encoder = TargetEncoder()
y_train_encoded = encoder.fit_transform(y_train, y_train)
y_test_encoded = encoder.transform(y_test, y_test)

# **Scaling**

In [144]:
scale_features = [
    'index',
    'County',
    'FIPS',
    'Met Objective of 45.5? (1)',
    'Age-Adjusted Death Rate',
    'Lower 95% Confidence Interval for Death Rate',
    'Upper 95% Confidence Interval for Death Rate',
    'Average Deaths per Year',
    'Recent 5-Year Trend (2) in Death Rates',
    'Lower 95% Confidence Interval for Trend',
    'Upper 95% Confidence Interval for Trend'
]

In [145]:
ss = StandardScaler()
X_train[scale_features] = ss.fit_transform(X_train[scale_features])
X_test[scale_features] = ss.transform(X_test[scale_features])

# **Model training**

In [146]:
lr = LinearRegression(fit_intercept=True)
lr.fit(X_train, y_train_encoded)

LinearRegression()

In [147]:
lr_pred = lr.predict(X_test)
lr_mse = mean_squared_error(y_test_encoded, lr_pred)
lr_rmse = np.sqrt(lr_mse)
lr_mae = mean_absolute_error(y_test_encoded, lr_pred)
lr_r2 = r2_score(y_test_encoded, lr_pred)

print(f"LinearRegression MSE:      {lr_mse:.4f}")
print(f"LinearRegression RMSE:     {lr_rmse:.4f}")
print(f"LinearRegression MAE:      {lr_mae:.4f}")
print(f"LinearRegression R2-Score: {lr_r2:.4f}")

LinearRegression MSE:      0.6951
LinearRegression RMSE:     0.8337
LinearRegression MAE:      0.6798
LinearRegression R2-Score: 0.5013


In [148]:
rf_params = {
    'n_estimators': [100, 250],
    'max_depth': [10, 20],
    'criterion': ["squared_error"]
}
rf_cv = GridSearchCV(RandomForestRegressor(n_jobs=-1), rf_params, cv=3).fit(X_train, y_train_encoded)
print(rf_cv.best_params_)

{'criterion': 'squared_error', 'max_depth': 20, 'n_estimators': 250}


In [149]:
rf = RandomForestRegressor(
    **rf_cv.best_params_,
    n_jobs=-1
)
rf.fit(X_train, y_train_encoded)

RandomForestRegressor(max_depth=20, n_estimators=250, n_jobs=-1)

In [150]:
rf_pred = rf.predict(X_test)
rf_mse = mean_squared_error(y_test_encoded, rf_pred)
rf_rmse = np.sqrt(rf_mse)
rf_mae = mean_absolute_error(y_test_encoded, rf_pred)
rf_r2 = r2_score(y_test_encoded, rf_pred)

print(f"RandomForestRegressor MSE:      {rf_mse:.4f}")
print(f"RandomForestRegressor RMSE:     {rf_rmse:.4f}")
print(f"RandomForestRegressor MAE:      {rf_mae:.4f}")
print(f"RandomForestRegressor R2-Score: {rf_r2:.4f}")

RandomForestRegressor MSE:      0.0219
RandomForestRegressor RMSE:     0.1481
RandomForestRegressor MAE:      0.0221
RandomForestRegressor R2-Score: 0.9843


In [151]:
svr_params = {
    'kernel': ['linear'],
    'C': [1.0]
}
svr_cv = GridSearchCV(SVR(), svr_params, cv=2).fit(X_train, y_train_encoded)
print(svr_cv.best_params_)

{'C': 1.0, 'kernel': 'linear'}


In [152]:
svr = SVR(
    **svr_cv.best_params_,
)
svr.fit(X_train, y_train_encoded)

SVR(kernel='linear')

In [153]:
svr_pred = svr.predict(X_test)
svr_mse = mean_squared_error(y_test_encoded, svr_pred)
svr_rmse = np.sqrt(svr_mse)
svr_mae = mean_absolute_error(y_test_encoded, svr_pred)
svr_r2 = r2_score(y_test_encoded, svr_pred)

print(f"SVR MSE:      {svr_mse:.4f}")
print(f"SVR RMSE:     {svr_rmse:.4f}")
print(f"SVR MAE:      {svr_mae:.4f}")
print(f"SVR R2-Score: {svr_r2:.4f}")

SVR MSE:      0.6758
SVR RMSE:     0.8221
SVR MAE:      0.5891
SVR R2-Score: 0.5152


# **Prediction phase**

In [154]:
input_data = [36061, 0.0, 60.1, 43.2, 81.9, 8.0, 1.2, -0.6, 3.0, 1.0, 0.0]
input_data_as_numpy_array = np.array(input_data)
input_data_reshaped = input_data_as_numpy_array.reshape(1,-1)
prediction = rf.predict(input_data_reshaped)
print(prediction)
if (prediction[0] == 0):
  print('falling')
else:
  print('stable')

[3.32333242]
stable


# **Pickling**

In [155]:
import pickle
from category_encoders import TargetEncoder

# Suppose y_train is your target variable used during training
encoder = TargetEncoder()
y_train_encoded = encoder.fit_transform(y_train, y_train)  # fit the encoder

# Save the trained encoder to a proper pickle file
with open('target_encoder.pkl', 'wb') as f:
    pickle.dump(encoder, f)


In [156]:
with open('scaler.pkl', 'rb') as f:
    ss = pickle.load(f)

In [157]:
with open('best_model.pkl','wb') as file:
  pickle.dump(rf,file)

# **Deployment phase**

In [158]:
import joblib
joblib.dump(rf, "cancer_model.pkl")

from google.colab import files
files.download("cancer_model.pkl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [159]:
X_train.columns

Index(['index', 'County', 'FIPS', 'Met Objective of 45.5? (1)',
       'Age-Adjusted Death Rate',
       'Lower 95% Confidence Interval for Death Rate',
       'Upper 95% Confidence Interval for Death Rate',
       'Average Deaths per Year', 'Recent 5-Year Trend (2) in Death Rates',
       'Lower 95% Confidence Interval for Trend',
       'Upper 95% Confidence Interval for Trend'],
      dtype='object')